In [1]:
import robot
import map
import solver
import utils

import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

import gtsam
from gtsam.symbol_shorthand import L, X

# unitTest 1: Move robot and check odometrey measurements

In [2]:
car = robot.robot()

#control inputs
dtheta = np.radians(20)
ds = 0.4

time = 0 #[s]
dt = 0.5 
timeFinal = 10

#init history loggers
hist_GT = np.array(car.pose)
hist_DR = hist_GT.copy()

#set graphics
fig , ax = utils.setWorldMap()
graphic_GT_traj, = plt.plot([], [],'ko-',markersize = 5)
graphic_DR_traj, = plt.plot([], [],'ro-',markersize = 5)
ax.legend(["ground truth","dead reckoning"])

#run and plot simulation
xcurrent_DR = hist_GT.copy()
with plt.ion():
    while time <= timeFinal:
        meas_odom = car.moveAndMeasureOdometrey((ds,ds,dtheta))
        
        #dead reckoning integration
        xcurrent_DR[2] += meas_odom.dtheta
        xcurrent_DR[0] += meas_odom.dx * np.cos(xcurrent_DR[2])
        xcurrent_DR[1] += meas_odom.dy * np.sin(xcurrent_DR[2])
        
        #log history
        hist_DR = np.vstack([hist_DR,xcurrent_DR])
        hist_GT = np.vstack([hist_GT,car.pose])

        #increase time
        time += dt

        #plot
        graphic_GT_traj.set_data(hist_GT[:,0],hist_GT[:,1])
        graphic_DR_traj.set_data(hist_DR[:,0],hist_DR[:,1])
        plt.draw()
        plt.pause(dt)

# unitTest 2: Build prior map

In [5]:
np.random.seed(seed=2)

N = 10 #number of landmarks
semantics = ("table","MEP","chair","pillar","clutter")
xrange = (-2,2)
yrange = (-1,3)
sigMin = -0.5
sigMax = 0.5

landmarks = [None] * N
for ii in range(N):
    sigs = np.random.uniform(low=sigMin, high=sigMax, size=(2,2))
    cov = sigs@sigs.T #enforce symmetric and positive definite: https://mathworld.wolfram.com/PositiveDefiniteMatrix.html
    landmarks[ii] = map.landmark(x = (np.random.rand()-0.5) * np.diff(xrange) + np.mean(xrange),
                                 y = (np.random.rand()-0.5) * np.diff(yrange) + np.mean(yrange),
                                 classLabel = np.random.choice(semantics),
                                 cov = cov)

priorMap = map.map(landmarks)
priorMap.plot(plotIndex = True,plotCov = True)

# unitTest 3: Move car and check landmark measurements

In [7]:
#------Build worldmap
fig , ax = utils.setWorldMap()
np.random.seed(seed=0)

N = 10 #number of landmarks
semantics = ("table","MEP","chair","pillar","clutter")
xrange = (-2,2)
yrange = (-1,3)

landmarks = [None] * N
for ii in range(N):
    landmarks[ii] = map.landmark(x = (np.random.rand()-0.5) * np.diff(xrange) + np.mean(xrange),
                                 y = (np.random.rand()-0.5) * np.diff(yrange) + np.mean(yrange),
                                 classLabel = np.random.choice(semantics),
                                 )
worldMap = map.map(landmarks)
worldMap.plot(ax = ax, plotIndex = True,plotCov = False)

#------Spawn Robot
car = robot.robot()
#control inputs
ds = 0.4 #[m]
dtheta = np.radians(20)
odom = (ds,ds,dtheta)

#------Simulation parameters
time = 0 #[s]
dt = 0.5 
timeFinal = 10

#init history loggers
hist_GT = np.array(car.pose)
hist_DR = hist_GT.copy()
hist_LM = []

#set graphics
graphic_GT_traj, = plt.plot([], [],'ko-',markersize = 5)
graphic_DR_traj, = plt.plot([], [],'ro-',markersize = 5)

#run and plot simulation
xcurrent_DR = hist_GT.copy()
with plt.ion():
    while time <= timeFinal:
        meas_odom = car.moveAndMeasureOdometrey(odom)

        meas_lm = car.measureLandmarks(worldMap)

        #dead reckoning integration
        xcurrent_DR[2] += meas_odom.dtheta
        xcurrent_DR[0] += meas_odom.dx * np.cos(xcurrent_DR[2])
        xcurrent_DR[1] += meas_odom.dy * np.sin(xcurrent_DR[2])
        
        #log history
        hist_GT = np.vstack([hist_GT,car.pose])
        hist_DR = np.vstack([hist_DR,xcurrent_DR])
        hist_LM.append(meas_lm)

        #increase time
        time += dt

        #plot
        graphic_GT_traj.set_data(hist_GT[:,0],hist_GT[:,1])
        graphic_DR_traj.set_data(hist_DR[:,0],hist_DR[:,1])
        plt.draw()
        plt.pause(dt)

print(hist_LM)

[[], [], [<robot.meas_landmark object at 0x7f5b6e3857f0>], [<robot.meas_landmark object at 0x7f5b6e553100>], [<robot.meas_landmark object at 0x7f5b6de4ca90>, <robot.meas_landmark object at 0x7f5b6de54df0>], [<robot.meas_landmark object at 0x7f5b6de4c3a0>, <robot.meas_landmark object at 0x7f5b6e553700>, <robot.meas_landmark object at 0x7f5b6de0b3d0>], [<robot.meas_landmark object at 0x7f5b6e2f2eb0>, <robot.meas_landmark object at 0x7f5b7c308c10>, <robot.meas_landmark object at 0x7f5b6de13970>], [<robot.meas_landmark object at 0x7f5b6ddfec40>, <robot.meas_landmark object at 0x7f5b6de138e0>], [<robot.meas_landmark object at 0x7f5b6de6c460>, <robot.meas_landmark object at 0x7f5b6de13a00>], [<robot.meas_landmark object at 0x7f5b7c308160>], [], [], [], [], [], [], [], [], [], [], [<robot.meas_landmark object at 0x7f5b741ea220>]]


# unitTest4: Plot Robot Cone

In [10]:
#------Build worldmap
fig , ax = utils.setWorldMap()
np.random.seed(seed=0)

#------Spawn Robot
car = robot.robot()
car.plot(ax)
#control inputs
ds = 0.4 #[m]
dtheta = np.radians(20)
odom = (ds,ds,dtheta)

#------Simulation parameters
time = 0 #[s]
dt = 0.5 
timeFinal = 10

with plt.ion():
    while time <= timeFinal:
        car.moveAndMeasureOdometrey(odom)

        #increase time
        time += dt

        car.plot()
        plt.draw()
        plt.pause(dt)

# unitTest5: putting everything together before gtsam

In [3]:
car, worldMap, ax, fig = utils.default_world()
prms = utils.default_parameters()

#init history loggers
hist_GT = np.array(car.pose)
hist_DR = hist_GT.copy()
hist_LM = []

#set graphics
graphic_GT_traj, = plt.plot([], [],'ko-',markersize = 1)
graphic_DR_traj, = plt.plot([], [],'ro-',markersize = 1)

#run and plot simulation
time = 0
xcurrent_DR = hist_GT.copy()
with plt.ion():
    while time <= prms["time"]["timeFinal"]:
        meas_odom = car.moveAndMeasureOdometrey(prms["odom"])

        meas_lm = car.measureLandmarks(worldMap)

        #dead reckoning integration
        xcurrent_DR[2] += meas_odom.dtheta
        xcurrent_DR[0] += meas_odom.dx * np.cos(xcurrent_DR[2])
        xcurrent_DR[1] += meas_odom.dy * np.sin(xcurrent_DR[2])

        #log history
        hist_GT = np.vstack([hist_GT,car.pose])
        hist_DR = np.vstack([hist_DR,xcurrent_DR])
        hist_LM.append(meas_lm)

        #increase time
        time += prms["time"]["dt"]

        #plot
        car.plot()
        graphic_GT_traj.set_data(hist_GT[:,0],hist_GT[:,1])
        graphic_DR_traj.set_data(hist_DR[:,0],hist_DR[:,1])
        plt.draw()
        plt.pause(prms["time"]["dt"])

for _ in hist_LM:
    print(_)

[<robot.meas_landmark object at 0x7f8284487cd0>, <robot.meas_landmark object at 0x7f82842197f0>, <robot.meas_landmark object at 0x7f82842198e0>, <robot.meas_landmark object at 0x7f8284219940>, <robot.meas_landmark object at 0x7f82842199a0>, <robot.meas_landmark object at 0x7f8284219a00>, <robot.meas_landmark object at 0x7f8284219a60>, <robot.meas_landmark object at 0x7f8284219ac0>]
[<robot.meas_landmark object at 0x7f8284219730>, <robot.meas_landmark object at 0x7f8284219be0>, <robot.meas_landmark object at 0x7f8284219ee0>, <robot.meas_landmark object at 0x7f8284219f70>, <robot.meas_landmark object at 0x7f8284219e80>, <robot.meas_landmark object at 0x7f8284219dc0>, <robot.meas_landmark object at 0x7f8284219fd0>]
[<robot.meas_landmark object at 0x7f8284219b50>, <robot.meas_landmark object at 0x7f82844554c0>, <robot.meas_landmark object at 0x7f8284455520>, <robot.meas_landmark object at 0x7f82841dac70>, <robot.meas_landmark object at 0x7f82841dab80>, <robot.meas_landmark object at 0x7f82

# unitTest6 GTSAM

notes: </br>
examples folder: /home/alon18/LocalInstalls/miniconda3/envs/smallslam/lib/python3.9/site-packages/gtsam/examples </br>
* imuFactorISAM2Example.py - understand difference between graph.add and graph.push_back </br>
  + never mind, they are the same (?) https://gtsam.org/doxygen/a03480.html#a1bd7154e74538b5934fc616bb5ef7281 </br>
* PlanarSLAMExample.py- factor for range and bearing </br>


from VisualISAMExample.py: </br>
      # If this is the first iteration, add a prior on the first pose to set the coordinate frame </br>
        # and a prior on the first landmark to set the scale </br>
        # Also, as iSAM solves incrementally, we must wait until each is observed at least twice before </br>
        # adding it to iSAM. </br>

In [4]:
car, worldMap, ax, fig = utils.default_world()
prms = utils.default_parameters()
backend = solver.solver()

#init history loggers
hist_GT = np.array(car.pose)
hist_DR = hist_GT.copy()
hist_LM = []
hist_solverResults = []

#set graphics
graphic_GT_traj, = plt.plot([], [],'ko-',markersize = 1)
graphic_DR_traj, = plt.plot([], [],'ro-',markersize = 1)

#run and plot simulation
time = 0
xcurrent_DR = hist_GT.copy()
with plt.ion():
    while time <= prms["time"]["timeFinal"]:
        meas_odom = car.moveAndMeasureOdometrey((prms["odom"]))
        meas_lms = car.measureLandmarks(worldMap)

        backend.addOdomMeasurement(meas_odom)
        #for meas_lm in meas_lms:
        #    backend.addlandmarkMeasurement(meas_lm)
        backend.update()

        #dead reckoning integration
        xcurrent_DR[2] += meas_odom.dtheta
        xcurrent_DR[0] += meas_odom.dx * np.cos(xcurrent_DR[2])
        xcurrent_DR[1] += meas_odom.dy * np.sin(xcurrent_DR[2])
        
        #log history
        hist_GT = np.vstack([hist_GT,car.pose])
        hist_DR = np.vstack([hist_DR,xcurrent_DR])
        hist_LM.append(meas_lm)
        hist_solverResults.append(backend.getResult())

        #increase time
        time += prms["time"]["dt"]

        #plot
        car.plot()
        graphic_GT_traj.set_data(hist_GT[:,0],hist_GT[:,1])
        graphic_DR_traj.set_data(hist_DR[:,0],hist_DR[:,1])
        plt.draw()
        plt.pause(prms["time"]["dt"])

RuntimeError: Attempting to add a key-value pair with key "x0", key already exists.

In [5]:
backend.graph

NonlinearFactorGraph: size: 2

Factor 0: PriorFactor on x0
  prior mean:  (0, 0, 0)
isotropic dim=3 sigma=0.1

Factor 1: BetweenFactor(x0,x1)
  measured:  (0.518246826, 0.40319746, 0.341316708)
  noise model: diagonal sigmas[0.1; 0.1; 0.0523598776];
